# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_URL = "https://books.toscrape.com/catalogue/"


# get all pages
def get_all_pages():
    urls = []
    page_number = 1

    for i in range(50):
        url = f"https://books.toscrape.com/catalogue/category/books_1/page-{page_number}.html"
        page_number += 1
        urls.append(url)
    return urls


#second level scrapping of book details
def get_book_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # UPC
    upc = soup.find('th', string='UPC').find_next_sibling('td').text.strip()

    # Genre
    genre = soup.find('ul', class_='breadcrumb').find_all('li')[-2].text.strip()

    return upc, genre


#main scrapping of + dataframe
def scrapping_books(url):
    books_data = []
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    books = soup.find_all('li', class_='col-xs-6 col-sm-4 col-md-3 col-lg-3')

    for book in books:
        title = book.find('h3').text.strip()
        price_text = book.find('div', class_='product_price').find('p', class_='price_color').text.strip()
        price = float(price_text.replace('£', '').strip())  # Convert price to float

        availability = book.find('div', class_='product_price').find('p', class_='instock availability').text.strip()

        # Rating
        rating_tag = book.find('p', class_='star-rating')
        rating_class = rating_tag['class']
        rating_text = rating_class[1]
        rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
        rating = rating_map.get(rating_text, 0)

        if rating < 4 or price > 20:
            continue

        # Page link
        relative_url = book.find('a')['href']
        book_url = BASE_URL + relative_url.replace('../../', '')

        # UPC and genre
        upc, genre = get_book_details(book_url)

        books_data.append({
            'title': title,
            'price': price,
            'availability': availability,
            'rating': rating,
            'upc': upc,
            'genre': genre
        })

    return pd.DataFrame(books_data)

def parse_all_books():
    pages = get_all_pages()
    all_book_df = pd.DataFrame()  # Initialize an empty DataFrame

    for page in pages:
        page_df = scrapping_books(url=page)  # Get the DataFrame for the current page
        all_book_df = pd.concat([all_book_df, page_df], ignore_index=True)  # Concatenate to the main DataFrame
        print(f'Scraped page: {page}')
    
    return all_book_df

# Run the script
all_book_df = parse_all_books()

Scraped page: https://books.toscrape.com/catalogue/category/books_1/page-1.html
Scraped page: https://books.toscrape.com/catalogue/category/books_1/page-2.html
Scraped page: https://books.toscrape.com/catalogue/category/books_1/page-3.html
Scraped page: https://books.toscrape.com/catalogue/category/books_1/page-4.html
Scraped page: https://books.toscrape.com/catalogue/category/books_1/page-5.html
Scraped page: https://books.toscrape.com/catalogue/category/books_1/page-6.html
Scraped page: https://books.toscrape.com/catalogue/category/books_1/page-7.html
Scraped page: https://books.toscrape.com/catalogue/category/books_1/page-8.html
Scraped page: https://books.toscrape.com/catalogue/category/books_1/page-9.html
Scraped page: https://books.toscrape.com/catalogue/category/books_1/page-10.html
Scraped page: https://books.toscrape.com/catalogue/category/books_1/page-11.html
Scraped page: https://books.toscrape.com/catalogue/category/books_1/page-12.html
Scraped page: https://books.toscrape.

In [9]:
all_book_df.head()

,title,price,availability,rating,upc,genre
0,Set Me Free,17.46,In stock,5,ce6396b0f23f6ecc,Young Adult
1,The Four Agreements: A ...,17.66,In stock,5,6258a1f6a6dcfe50,Spirituality
2,Sophie's World,15.94,In stock,5,6be3beb0793a53e7,Philosophy
3,Untitled Collection: Sabbath Poems ...,14.27,In stock,4,657fe5ead67a7767,Poetry
4,This One Summer,19.49,In stock,4,51653ef291ab7ddc,Sequential Art
